In [1]:
from spectacle import *

Manim Community v0.14.0

Welcome to spectacle


In [22]:
%%manim -v WARNING --disable_caching -qm Test

class Test(Scene):
    def construct(self):
        stopwatch = StopWatch()
        self.add(stopwatch)
        
        stopwatch.start_running()
        self.wait(4)
        


class StopWatch(VGroup):
    def __init__(self, center=ORIGIN, primary_color=WHITE, bg_color=BLACK):
        self.main_center = center
        self.outer_radius = 1.5
        self.inner_radius = 1.25
        self.primary_color=primary_color
        self.bg_color=bg_color
        self.outer_circle = Circle(
            radius=self.outer_radius, fill_opacity=1, color=primary_color
        ).move_to(center)
        self.inner_circle = Circle(
            radius=self.inner_radius, fill_opacity=1, color=bg_color
        ).move_to(center)
        self.vtime = 0

        self.ticks = []
        for n in range(8):
            angle = n * 2 * PI / 8
            vec = point(np.cos(angle), np.sin(angle))
            start = self.main_center + 0.8 * vec * self.inner_radius
            end = self.main_center + 0.9 * vec * self.inner_radius
            tick_line = Line(start=start, end=end, stroke_width=DEFAULT_STROKE_WIDTH / 2)
            self.ticks.append(tick_line)

        self.center_axle = Dot(point=center)
        self.ticker_line = Line(start=center, end=center + 0.75 * UP)
        self.ticker = Polygon(
            center + 0.75 * self.inner_radius,
            center + (DEFAULT_DOT_RADIUS / 3) * RIGHT,
            center + (DEFAULT_DOT_RADIUS / 3) * LEFT,
            fill_opacity=1,
        )
        
        self.ghost_ticker = DashedLine(self.main_center, self.main_center+0.75*self.inner_radius*UP, color=bg_color)
        self.time_sector = AnnularSector(inner_radius=self.inner_radius*0.5,outer_radius=self.inner_radius*0.75,angle=self.ticker_line.get_angle(), start_angle=PI/2, color=bg_color)

        vmobjects = [
            self.outer_circle,
            self.inner_circle,
            self.ghost_ticker,
            *self.ticks,
            self.time_sector,
            self.center_axle,
            self.ticker,
        ]
        super().__init__(*vmobjects)
        
    def update_ticker(self,dt):
        new_time = self.vtime+dt
        self.ticker_line.rotate(angle=-dt, about_point=self.main_center)
        self.ticker.rotate(angle=-dt, about_point=self.main_center)
        self.time_sector.become(AnnularSector(inner_radius=self.inner_radius*0.5,outer_radius=self.inner_radius*0.75,angle=self.ticker_line.get_angle()-3*PI/4, start_angle=PI/2, color=RED))
        if self.vtime == 0:
            self.ghost_ticker.set_color(self.primary_color)
        self.vtime = new_time
        
        
    def start_running(self):
        self.add_updater(lambda _,dt: self.update_ticker(dt))
        
